In [1]:
from ray.rllib.policy import Policy
import ray
import inspect
import time
from statistics import mean, stdev
from CybORG import CybORG
from CybORG.Agents import B_lineAgent, SleepAgent, GreenAgent
from CybORG.Agents.SimpleAgents.BaseAgent import BaseAgent
from CybORG.Agents.SimpleAgents.BlueReactAgent import BlueReactRemoveAgent
from CybORG.Agents.SimpleAgents.Meander import RedMeanderAgent
from CybORG.Agents.Wrappers.EnumActionWrapper import EnumActionWrapper
from CybORG.Agents.Wrappers.FixedFlatWrapper import FixedFlatWrapper
from CybORG.Agents.Wrappers.OpenAIGymWrapper import OpenAIGymWrapper
from CybORG.Agents.Wrappers.ReduceActionSpaceWrapper import ReduceActionSpaceWrapper
from CybORG.Agents.Wrappers import ChallengeWrapper
import os
import tensorflow as tf
import numpy as np
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents import ppo
from ray.tune.registry import register_env
from CybORG.Agents.Wrappers.rllib_wrapper import RLlibWrapper
import warnings
import gym
warnings.filterwarnings('ignore')

Option Critic Model

Additional Option Critic functions

In [2]:
import inspect
import time
from statistics import mean, stdev
from CybORG import CybORG
from CybORG.Agents import B_lineAgent, SleepAgent, GreenAgent
from CybORG.Agents.SimpleAgents.BaseAgent import BaseAgent
from CybORG.Agents.SimpleAgents.BlueReactAgent import BlueReactRemoveAgent
from CybORG.Agents.SimpleAgents.Meander import RedMeanderAgent
from CybORG.Agents.Wrappers.EnumActionWrapper import EnumActionWrapper
from CybORG.Agents.Wrappers.FixedFlatWrapper import FixedFlatWrapper
from CybORG.Agents.Wrappers.OpenAIGymWrapper import OpenAIGymWrapper
from CybORG.Agents.Wrappers.ReduceActionSpaceWrapper import ReduceActionSpaceWrapper
from CybORG.Agents.Wrappers import ChallengeWrapper
import os
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents import ppo
from ray.tune.registry import register_env
from CybORG.Agents.Wrappers.rllib_wrapper import RLlibWrapper
import warnings
warnings.filterwarnings('ignore')

In [3]:
MAX_EPS = 50
agent_name = 'Blue'

def wrap(env):
    return RLlibWrapper(agent_name="Blue", env=env)


def evaluate(steps):
    path = str(inspect.getfile(CybORG))
    path = path[:-10] + '/Shared/Scenarios/Scenario1b.yaml'

    #print(f'using CybORG v{cyborg_version}, {scenario}\n')
    for num_steps in steps:
        #for red_agent in [B_lineAgent, RedMeanderAgent, SleepAgent]:
        rs = []
        for red_agent in [RedMeanderAgent, B_lineAgent, SleepAgent]:

            cyborg = CybORG(path, 'sim', agents={'Red': red_agent})
            wrapped_cyborg = wrap(cyborg)

            observation = wrapped_cyborg.reset()
            # observation = cyborg.reset().observation

            action_space = wrapped_cyborg.get_action_space(agent_name)
            # action_space = cyborg.get_action_space(agent_name)
            total_reward = []
            actions = []
            for i in range(MAX_EPS):
                r = []
                a = []
                # cyborg.env.env.tracker.render()
                for j in range(num_steps):
                    action = agent.compute_single_action(observation)
                    #action = agent.get_action(observation, action_space)
                    observation, rew, done, info = wrapped_cyborg.step(action)
                    # result = cyborg.step(agent_name, action)
                    r.append(rew)
                    # r.append(result.reward)
                    a.append((str(cyborg.get_last_action('Blue')), str(cyborg.get_last_action('Red'))))
                total_reward.append(sum(r))
                actions.append(a)
                # observation = cyborg.reset().observation
                observation = wrapped_cyborg.reset()
            rs.append(mean(total_reward))
            print(f'Average reward for red agent {red_agent.__name__} and steps {num_steps} is: {mean(total_reward):.1f} with a standard deviation of {stdev(total_reward):.1f}')
    return rs

In [4]:
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents import ppo
    
class MultiEnv(gym.Env):
    def __init__(self, env_config):
        # pick actual env based on worker and env indexes
        self.env = self.choose_env_for(env_config.worker_index)
        self.action_space = self.env.action_space
        self.observation_space = self.env.observation_space
    def reset(self):
        return self.env.reset()
    def step(self, action):
        return self.env.step(action)   
    def choose_env_for(self, index):
        if index > 20:
            path = str(inspect.getfile(CybORG))
            path = path[:-10] + '/Shared/Scenarios/Scenario2.yaml'
            agents = {"Red": SleepAgent, "Green": GreenAgent}
            cyborg = CybORG(scenario_file=path, environment='sim', agents=agents)
            return RLlibWrapper(env=cyborg, agent_name="Blue", max_steps=100)
        elif index % 2 == 0:
            path = str(inspect.getfile(CybORG))
            path = path[:-10] + '/Shared/Scenarios/Scenario2.yaml'
            agents = {"Red": B_lineAgent, "Green": GreenAgent}
            cyborg = CybORG(scenario_file=path, environment='sim', agents=agents)
            return RLlibWrapper(env=cyborg, agent_name="Blue", max_steps=100)
        else:
            path = str(inspect.getfile(CybORG))
            path = path[:-10] + '/Shared/Scenarios/Scenario2.yaml'
            agents = {"Red": RedMeanderAgent, "Green": GreenAgent}
            cyborg = CybORG(scenario_file=path, environment='sim', agents=agents)
            return RLlibWrapper(env=cyborg, agent_name="Blue", max_steps=100)
        
def print_results(results_dict):
    train_iter = results_dict["training_iteration"]
    r_mean = results_dict["episode_reward_mean"]
    r_max = results_dict["episode_reward_max"]
    r_min = results_dict["episode_reward_min"]
    print(f"{train_iter:4d} \tr_mean: {r_mean:.1f} \tr_max: {r_max:.1f} \tr_min: {r_min: .1f}")
    
register_env("multienv", lambda config: MultiEnv(config))

batch_size = 4000

all_rewards = []; all_evals = [];
config = { "env": "multienv",
                "num_gpus": 1,"num_workers": 20,
                'num_cpus_per_worker':1,
                #"log_level": 'DEBUG',
                "model": {
                    "fcnet_hiddens": [512, 512],
                    "fcnet_activation": "relu",
                },
                "framework": "tf2",
                "train_batch_size": batch_size,
                "horizon": 100,
                "gamma": 0.95,
            } 
trainer = ppo.PPOTrainer(config=config, env="multienv")
reward = []
m = -80
for i in range(int(1000)):
    results_dict=trainer.train()
    print_results(results_dict)
    if m < results_dict["episode_reward_mean"]:
        reward.append(results_dict["episode_reward_mean"])
        print(trainer.save("multisingle_ppo"))
        m = results_dict["episode_reward_mean"]
        

(RolloutWorker pid=3270) 2022-08-11 11:04:16,405	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=3276) 2022-08-11 11:04:16,513	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=3273) 2022-08-11 11:04:16,628	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=3272) 2022-08-11 11:04:16,655	WAR

   1 	r_mean: -648.8 	r_max: -196.3 	r_min: -1148.8
   2 	r_mean: -550.2 	r_max: -196.3 	r_min: -1148.8
   3 	r_mean: -448.8 	r_max: -116.8 	r_min: -1146.2
   4 	r_mean: -373.8 	r_max: -116.8 	r_min: -1109.4
   5 	r_mean: -322.5 	r_max: -73.7 	r_min: -1086.8
   6 	r_mean: -299.8 	r_max: -73.7 	r_min: -1035.3
   7 	r_mean: -264.1 	r_max: -134.1 	r_min: -934.2
   8 	r_mean: -251.5 	r_max: -64.8 	r_min: -934.2
   9 	r_mean: -249.1 	r_max: -64.8 	r_min: -629.8
  10 	r_mean: -245.2 	r_max: -126.6 	r_min: -629.8
  11 	r_mean: -237.7 	r_max: -111.4 	r_min: -742.6
  12 	r_mean: -234.5 	r_max: -111.4 	r_min: -742.6
  13 	r_mean: -230.1 	r_max: -126.0 	r_min: -742.6
  14 	r_mean: -213.8 	r_max: -122.7 	r_min: -419.3
  15 	r_mean: -207.5 	r_max: -122.7 	r_min: -419.3
  16 	r_mean: -204.9 	r_max: -61.8 	r_min: -419.3
  17 	r_mean: -197.0 	r_max: -39.8 	r_min: -323.0
  18 	r_mean: -193.0 	r_max: -39.8 	r_min: -640.2
  19 	r_mean: -189.4 	r_max: -93.7 	r_min: -640.2
  20 	r_mean: -178.8 	r_max: -70.

 146 	r_mean: -42.6 	r_max: -10.8 	r_min: -148.7
multisingle_ppo/checkpoint_000146/checkpoint-146
 147 	r_mean: -42.0 	r_max: -10.7 	r_min: -141.7
multisingle_ppo/checkpoint_000147/checkpoint-147
 148 	r_mean: -43.5 	r_max: -10.7 	r_min: -141.7
 149 	r_mean: -44.9 	r_max: -10.8 	r_min: -141.7
 150 	r_mean: -41.1 	r_max: -9.7 	r_min: -106.4
multisingle_ppo/checkpoint_000150/checkpoint-150
 151 	r_mean: -42.8 	r_max: -9.7 	r_min: -219.4
 152 	r_mean: -46.2 	r_max: -10.3 	r_min: -219.4
 153 	r_mean: -46.7 	r_max: -11.2 	r_min: -169.7
 154 	r_mean: -48.7 	r_max: -11.2 	r_min: -169.7
 155 	r_mean: -51.2 	r_max: -11.9 	r_min: -137.7
 156 	r_mean: -50.2 	r_max: -11.8 	r_min: -142.8
 157 	r_mean: -48.1 	r_max: -10.9 	r_min: -142.8
 158 	r_mean: -49.3 	r_max: -10.7 	r_min: -142.0
 159 	r_mean: -52.2 	r_max: -10.7 	r_min: -158.7
 160 	r_mean: -51.6 	r_max: -9.6 	r_min: -158.7
 161 	r_mean: -51.6 	r_max: -9.6 	r_min: -152.4
 162 	r_mean: -50.6 	r_max: -10.7 	r_min: -159.7
 163 	r_mean: -53.5 	r_m

 310 	r_mean: -43.9 	r_max: -9.7 	r_min: -275.8
 311 	r_mean: -46.3 	r_max: -8.9 	r_min: -275.8
 312 	r_mean: -43.1 	r_max: -8.9 	r_min: -161.8
 313 	r_mean: -42.7 	r_max: -9.8 	r_min: -161.8
 314 	r_mean: -43.8 	r_max: -10.0 	r_min: -241.8
 315 	r_mean: -44.7 	r_max: -10.0 	r_min: -241.8
 316 	r_mean: -39.3 	r_max: -10.1 	r_min: -135.5
 317 	r_mean: -41.5 	r_max: -10.1 	r_min: -162.4
 318 	r_mean: -44.2 	r_max: -6.3 	r_min: -215.0
 319 	r_mean: -52.2 	r_max: -6.3 	r_min: -441.8
 320 	r_mean: -54.4 	r_max: -6.3 	r_min: -441.8
 321 	r_mean: -62.3 	r_max: -8.1 	r_min: -441.8
 322 	r_mean: -50.9 	r_max: -9.0 	r_min: -157.8
 323 	r_mean: -42.6 	r_max: -9.0 	r_min: -150.8
 324 	r_mean: -38.5 	r_max: -9.0 	r_min: -124.1
 325 	r_mean: -43.1 	r_max: -9.8 	r_min: -206.8
 326 	r_mean: -45.0 	r_max: -10.5 	r_min: -206.8
 327 	r_mean: -47.0 	r_max: -10.2 	r_min: -206.8
 328 	r_mean: -48.5 	r_max: -10.2 	r_min: -167.7
 329 	r_mean: -46.1 	r_max: -10.2 	r_min: -167.7
 330 	r_mean: -44.4 	r_max: -9.6

 475 	r_mean: -41.8 	r_max: -10.5 	r_min: -150.8
 476 	r_mean: -40.9 	r_max: -10.5 	r_min: -150.8
 477 	r_mean: -42.6 	r_max: -9.5 	r_min: -148.8
 478 	r_mean: -40.8 	r_max: -9.5 	r_min: -148.8
 479 	r_mean: -41.1 	r_max: -9.5 	r_min: -148.8
 480 	r_mean: -39.6 	r_max: -9.8 	r_min: -139.7
 481 	r_mean: -40.6 	r_max: -9.8 	r_min: -226.8
 482 	r_mean: -44.2 	r_max: -10.4 	r_min: -226.8
 483 	r_mean: -47.5 	r_max: -10.4 	r_min: -226.8
 484 	r_mean: -49.7 	r_max: -10.3 	r_min: -216.8
 485 	r_mean: -53.0 	r_max: -10.3 	r_min: -216.8
 486 	r_mean: -56.2 	r_max: -10.8 	r_min: -216.8
 487 	r_mean: -47.0 	r_max: -9.8 	r_min: -187.0
 488 	r_mean: -40.9 	r_max: -9.8 	r_min: -138.8
 489 	r_mean: -43.6 	r_max: -9.8 	r_min: -138.8
 490 	r_mean: -45.0 	r_max: -11.1 	r_min: -142.4
 491 	r_mean: -44.5 	r_max: -10.9 	r_min: -145.8
 492 	r_mean: -45.7 	r_max: -9.8 	r_min: -227.1
 493 	r_mean: -51.0 	r_max: -9.8 	r_min: -227.1
 494 	r_mean: -47.8 	r_max: -9.5 	r_min: -227.1
 495 	r_mean: -45.2 	r_max: -9.


KeyboardInterrupt



In [5]:
m = -40
for i in range(int(460)):
    results_dict=trainer.train()
    print_results(results_dict)
    if m < results_dict["episode_reward_mean"]:
        reward.append(results_dict["episode_reward_mean"])
        print(trainer.save("multisingle_ppo"))
        m = results_dict["episode_reward_mean"]

 542 	r_mean: -43.4 	r_max: -10.4 	r_min: -145.8
 543 	r_mean: -41.2 	r_max: -10.4 	r_min: -145.8
 544 	r_mean: -39.6 	r_max: -9.8 	r_min: -136.3
multisingle_ppo/checkpoint_000544/checkpoint-544
 545 	r_mean: -42.4 	r_max: -9.8 	r_min: -136.8
 546 	r_mean: -43.1 	r_max: -9.8 	r_min: -139.7
 547 	r_mean: -43.3 	r_max: -9.8 	r_min: -139.7
 548 	r_mean: -45.6 	r_max: -10.8 	r_min: -194.1
 549 	r_mean: -52.2 	r_max: -11.3 	r_min: -194.1
 550 	r_mean: -54.6 	r_max: -11.3 	r_min: -194.1
 551 	r_mean: -51.4 	r_max: -10.7 	r_min: -176.7
 552 	r_mean: -50.1 	r_max: -10.7 	r_min: -243.5
 553 	r_mean: -47.3 	r_max: -10.3 	r_min: -243.5
 554 	r_mean: -47.5 	r_max: -10.3 	r_min: -243.5
 555 	r_mean: -49.3 	r_max: -10.3 	r_min: -242.8
 556 	r_mean: -53.1 	r_max: -9.7 	r_min: -242.8
 557 	r_mean: -56.7 	r_max: -9.7 	r_min: -242.8
 558 	r_mean: -53.4 	r_max: -9.7 	r_min: -157.7
 559 	r_mean: -51.3 	r_max: -9.1 	r_min: -157.7
 560 	r_mean: -49.9 	r_max: -9.1 	r_min: -157.7
 561 	r_mean: -46.8 	r_max: -

 707 	r_mean: -46.1 	r_max: -10.8 	r_min: -151.8
 708 	r_mean: -46.9 	r_max: -10.8 	r_min: -187.3
 709 	r_mean: -49.9 	r_max: -10.3 	r_min: -229.8
 710 	r_mean: -53.6 	r_max: -10.3 	r_min: -229.8
 711 	r_mean: -48.6 	r_max: -10.3 	r_min: -229.8
 712 	r_mean: -45.5 	r_max: -10.3 	r_min: -165.5
 713 	r_mean: -40.8 	r_max: -9.1 	r_min: -154.8
 714 	r_mean: -43.0 	r_max: -9.1 	r_min: -167.8
 715 	r_mean: -38.9 	r_max: -9.1 	r_min: -167.8
 716 	r_mean: -39.6 	r_max: -9.8 	r_min: -159.0
 717 	r_mean: -47.8 	r_max: -9.8 	r_min: -159.0
 718 	r_mean: -45.0 	r_max: -9.8 	r_min: -159.0
 719 	r_mean: -46.3 	r_max: -9.8 	r_min: -151.8
 720 	r_mean: -45.1 	r_max: -9.8 	r_min: -151.8
 721 	r_mean: -45.8 	r_max: -9.8 	r_min: -177.8
 722 	r_mean: -44.4 	r_max: -9.8 	r_min: -177.8
 723 	r_mean: -41.9 	r_max: -10.4 	r_min: -149.8
 724 	r_mean: -45.5 	r_max: -10.7 	r_min: -152.8
 725 	r_mean: -48.3 	r_max: -10.7 	r_min: -152.8
 726 	r_mean: -49.4 	r_max: -10.7 	r_min: -240.7
 727 	r_mean: -49.9 	r_max: -1

 877 	r_mean: -46.7 	r_max: -10.8 	r_min: -166.8
 878 	r_mean: -43.5 	r_max: -10.7 	r_min: -148.8
 879 	r_mean: -44.5 	r_max: -10.7 	r_min: -148.8
 880 	r_mean: -48.3 	r_max: -10.7 	r_min: -149.8
 881 	r_mean: -47.4 	r_max: -10.8 	r_min: -149.8
 882 	r_mean: -47.9 	r_max: -10.8 	r_min: -168.8
 883 	r_mean: -50.1 	r_max: -10.8 	r_min: -168.8
 884 	r_mean: -50.9 	r_max: -10.4 	r_min: -205.8
 885 	r_mean: -54.9 	r_max: -10.4 	r_min: -245.7
 886 	r_mean: -51.3 	r_max: -9.7 	r_min: -245.7
 887 	r_mean: -49.5 	r_max: -9.7 	r_min: -245.7
 888 	r_mean: -43.0 	r_max: -9.7 	r_min: -166.8
 889 	r_mean: -49.8 	r_max: -10.7 	r_min: -166.8
 890 	r_mean: -50.4 	r_max: -10.7 	r_min: -210.7
 891 	r_mean: -52.1 	r_max: -10.7 	r_min: -210.7
 892 	r_mean: -47.5 	r_max: -10.8 	r_min: -210.7
 893 	r_mean: -51.2 	r_max: -11.2 	r_min: -165.8
 894 	r_mean: -53.5 	r_max: -9.7 	r_min: -217.8
 895 	r_mean: -54.9 	r_max: -9.7 	r_min: -217.8
 896 	r_mean: -52.9 	r_max: -10.7 	r_min: -217.8
 897 	r_mean: -53.1 	r_ma

In [ ]:
np.save('singlemulti_reward.npy', np.array(reward))

In [ ]:
1172 	r_mean: -59.6 	r_max: -12.5 	r_min: -121.7
multisingle_ppo/checkpoint_001172/checkpoint-1172

1074 	r_mean: -62.0 	r_max: -10.8 	r_min: -153.2
multisingle_ppo/checkpoint_001074/checkpoint-1074